## Import libraries

In [ ]:
! pip install -U lightautoml

In [ ]:
import gc
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import log_loss
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import QuantileTransformer

from lightautoml.tasks import Task
from lightautoml.automl.presets.tabular_presets import TabularUtilizedAutoML

## Prepare data for model training

In [ ]:
with open("../input/tps-june-data-preprocess/TPS_June_Dataset_Set1.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

Ytrain_oh = pd.get_dummies(train_df['target']).values

del processed_data
gc.collect()

## Build and validate the model

In [ ]:
FOLD = 5
N_THREADS = 4
TIMEOUT = 60 * 60 * 3

model = TabularUtilizedAutoML(
    task = Task('multiclass',), 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    general_params = {'use_algos': [['lgb', 'cb', 'cb_tuned'], ['linear_l2', 'cb']]},
    reader_params = {'n_jobs': N_THREADS, 'cv': FOLD},
)

y_pred_meta_lama = model.fit_predict(train_df, roles={'target':'target'})
print("\n\ny_pred_meta_lama: {}".format(y_pred_meta_lama.shape))

In [ ]:
oof_score = log_loss(Ytrain_oh, y_pred_meta_lama.data)
print("Aggregate OOF Score: {}".format(oof_score))

In [ ]:
y_pred_final_lama = model.predict(test_df)

In [ ]:
np.savez_compressed('./LAMA_Meta_Features.npz',
                    y_pred_meta_lama=y_pred_meta_lama.data, 
                    oof_score=oof_score,
                    y_pred_final_lama=y_pred_final_lama.data)

## Create submission file

In [ ]:
p_min = 0.025
p_max = 1 - p_min

#y_pred_final_lama = np.clip(y_pred_final_lama.data, p_min, p_max)
y_pred_final_lama = y_pred_final_lama.data

test_df = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")
submit_df = pd.DataFrame()
submit_df['id'] = test_df['id']
submit_df['Class_1'] = y_pred_final_lama[:,1]
submit_df['Class_2'] = y_pred_final_lama[:,2]
submit_df['Class_3'] = y_pred_final_lama[:,3]
submit_df['Class_4'] = y_pred_final_lama[:,4]
submit_df['Class_5'] = y_pred_final_lama[:,5]
submit_df['Class_6'] = y_pred_final_lama[:,6]
submit_df['Class_7'] = y_pred_final_lama[:,7]
submit_df['Class_8'] = y_pred_final_lama[:,8]
submit_df['Class_9'] = y_pred_final_lama[:,0]
submit_df.head()

In [ ]:
submit_df.to_csv("./LAMA_submission.csv", index=False)